## Import dependencies

In [1]:
import collections
from itertools import combinations, islice
from operator import attrgetter
from random import shuffle

## Methods

In [2]:
def create_deck():
    '''Method to create a standard 52-card deck (not shuffled).
    Returns a deque of namedtuples.'''
    
    Card = collections.namedtuple('Card', ['rank', 'suit'])
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    return collections.deque([Card(rank, suit) for suit in suits
                                               for rank in ranks])


def shuffle_deck(deck):
    '''Method to randomly shuffle the cards in a deck.
    Returns a deque of namedtuples.'''
    
    shuffle(deck)
    return deck


def deal_player_hands(deck, cards=2, players=5):
    '''Deals out hole cards to each player.  By default,
    2 hole cards are dealt to 5 players.
    This method also gives a colloquial name to each pair
    of hole cards, for later use in data analysis.
    A list containing a list of namedtuples of hole cards and
    the remaining cards in the deck is returned.
    '''
    
    player_hands = collections.defaultdict(list)
    for card in range(2): # Players each get 2 hole cards
        for player in range(1, players + 1):
            player_hands[player].append(deck.popleft())
                
    # get colloquial name for hole cards
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    for player in player_hands.keys():
        if player_hands[player][0].rank == player_hands[player][1].rank:
            player_hands[player].append('pair')
        elif ((player_hands[player][0].suit == player_hands[player][1].suit) and
             (abs(ranks.index(player_hands[player][0].rank) - ranks.index(player_hands[player][1].rank)) == 1) or
             (abs(ranks.index(player_hands[player][0].rank) - ranks.index(player_hands[player][1].rank)) == 12)):
            player_hands[player].append('suited connectors')
        elif player_hands[player][0].suit == player_hands[player][1].suit:
            player_hands[player].append('suited')
        elif ((abs(ranks.index(player_hands[player][0].rank) - ranks.index(player_hands[player][1].rank)) == 1) or
              (abs(ranks.index(player_hands[player][0].rank) - ranks.index(player_hands[player][1].rank)) == 12)):
            player_hands[player].append('connectors')
        else:
            high_value = max(ranks.index(player_hands[player][0].rank), ranks.index(player_hands[player][1].rank))
            player_hands[player].append(ranks[high_value] + '-high')
            
    PlayerInfo = collections.namedtuple('PlayerHand', ['player', 'cards', 'name']) 
    player_info = [PlayerInfo(player, player_hands[player][0:-1], player_hands[player][-1]) for player in player_hands.keys()]
    return [player_info, deck]


def deal_communal_cards(deck):
    '''Deals out the 5 communal cards.
    Returns a list of namedtuples.'''
    
    first_three = list(islice(deck, 1, 4))
    first_three.append(deck[5])
    first_three.append(deck[7])
    return first_three


def score_hand(hand):
    '''Assigns a numeric value to poker hands, higher being better.
    `hand` should be a list of five cards.
    Returns a namedtuple.''' 
    
    def is_flush(suit_dict):
        return len(suit_dict) == 1
    
    def is_straight(numeral_dict):
        if len(numeral_dict) != 5:
            return False
        else:
            ranks = [str(n) for n in range(2, 11)] + list('JQKA')
            min_numeral = min([ranks.index(x) for x in numeral_dict.keys()])
            max_numeral = max([ranks.index(x) for x in numeral_dict.keys()])
            if int(max_numeral) - int(min_numeral) == 4:
                return True
            low_straight = set(("A", "2", "3", "4", "5"))
            if not set(numeral_dict.keys()).difference(low_straight):
                return True
    
    def is_straight_flush(numeral_dict, suit_dict):
        return is_straight(numeral_dict) and is_flush(suit_dict)
    
    def is_royal_flush(numeral_dict, suit_dict):
        return is_straight_flush(suit_dict, numeral_dict) and 'A' in numeral_dict.keys() and '10' in numeral_dict.keys()
    
    def is_four_of_a_kind(numeral_dict):
        return 4 in numeral_dict.values()
    
    def is_three_of_a_kind(numeral_dict):
        return 3 in numeral_dict.values()
    
    def is_full_house(numeral_dict):
        return len(numeral_dict) == 2 and 3 in numeral_dict.values() and 2 in numeral_dict.values()
    
    def is_two_pair(numeral_dict):
        return len(numeral_dict) == 3 and 2 in numeral_dict.values() and 1 in numeral_dict.values()
    
    def is_pair(numeral_dict):
        return 2 in numeral_dict.values()
        
    numeral_dict = collections.defaultdict(int)
    suit_dict = collections.defaultdict(int)
    for card in hand:
        numeral_dict[card.rank] += 1
        suit_dict[card.suit] += 1
        
    hand_values = collections.namedtuple('HandValue', ['name', 'value', 'kickers', 'high', 'low', 'cards'])
    
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    highest_rank = ranks[max([ranks.index(rank) for rank in numeral_dict.keys()])]
    lowest_rank = ranks[min([ranks.index(rank) for rank in numeral_dict.keys()])]
    
    if is_royal_flush(numeral_dict, suit_dict):
        return hand_values('royal flush', 10, None, highest_rank, lowest_rank, hand)
    elif is_straight_flush(numeral_dict, suit_dict):
        return hand_values('straight flush', 9, None, highest_rank, lowest_rank, hand)
    elif is_four_of_a_kind(numeral_dict):
        # `High` for four-of-a-kind corresponds to the set of 4
        kicker = [rank for rank in numeral_dict.keys() if numeral_dict[rank] == 1]
        high = [rank for rank in numeral_dict.keys() if numeral_dict[rank] == 4]
        return hand_values('4-of-a-kind', 8, kicker, high[0], lowest_rank, hand)
    elif is_full_house(numeral_dict):
        # `High` for a full house corresponds to the set of 3
        high = [rank for rank in numeral_dict.keys() if numeral_dict[rank] == 3]
        low = [rank for rank in numeral_dict.keys() if numeral_dict[rank] == 2]
        return hand_values('full house', 7, None, high[0], low[0], hand)
    elif is_flush(suit_dict):
        # 'kickers' are all cards, sorted from highest to lowest value
        kicker_indices = sorted([ranks.index(rank) for rank in numeral_dict.keys()], reverse=True)
        return hand_values('flush', 6, [ranks[index] for index in kicker_indices], highest_rank, lowest_rank, hand)
    elif is_straight(numeral_dict):
        return hand_values('straight', 5, None, highest_rank, lowest_rank, hand)
    elif is_three_of_a_kind(numeral_dict):
        # `High` for three-of-a-kind corresponds to the set of 3
        high = [rank for rank in numeral_dict.keys() if numeral_dict[rank] == 3]
        kicker_indices = sorted([ranks.index(rank) for rank in numeral_dict.keys() if numeral_dict[rank] != 3], reverse=True)
        return hand_values('3-of-a-kind', 4, [ranks[index] for index in kicker_indices], high[0], lowest_rank, hand)
    elif is_two_pair(numeral_dict):
        # `High` for two pair is the rank of the highest pair
        # `Low` for two pair is the rank of the lowest pair
        pair_indices = [ranks.index(rank) for rank in numeral_dict.keys() if numeral_dict[rank] == 2]
        kicker = [rank for rank in numeral_dict.keys() if numeral_dict[rank] == 1]
        return hand_values('2-pair', 3, kicker, ranks[max(pair_indices)], ranks[min(pair_indices)], hand)
    elif is_pair(numeral_dict):
        # `High` for a pair is the rank of the pair
        kicker_indices = sorted([ranks.index(rank) for rank in numeral_dict.keys() if numeral_dict[rank] != 2], reverse=True)
        pair_index = [ranks.index(rank) for rank in numeral_dict.keys() if numeral_dict[rank] == 2][0]
        return hand_values('pair', 2, [ranks[index] for index in kicker_indices], ranks[pair_index], lowest_rank, hand)
    else:
        # 'kickers' are all cards, sorted from highest to lowest value
        kicker_indices = sorted([ranks.index(rank) for rank in numeral_dict.keys()], reverse=True)
        return hand_values('high card', 1, [ranks[index] for index in kicker_indices], highest_rank, lowest_rank, hand) 

    
def find_top_hand_for_player(hands):
    '''Takes in a list of hands and ranks them from best to worst'''
    hand_scores = [score_hand(hand) for hand in hands]
    sorted_hands = sorted(hand_scores, key=attrgetter('value'), reverse=True)
    top_hands = [hand for hand in sorted_hands if hand.value == sorted_hands[0].value]
    # No extra work needed if only 1 top hand is present.
    # (Or if there are multiple royal flush or 4-of-a-kind top hands, which actually can't happen.)
    if ((len(top_hands) == 1) or (top_hands[0].name == 'royal flush')
        or (top_hands[0].name == '4-of-a-kind')):
        return top_hands[0]
    else:
        ranks = [str(n) for n in range(2, 11)] + list('JQKA')
        hand_type = top_hands[0].name
        if 'straight' in hand_type:
            high_card_indices = sorted([ranks.index(hand.high) for hand in top_hands], reverse=True)
            return [hand for hand in top_hands if hand.high == ranks[high_card_indices[0]]][0]
        elif (hand_type == 'flush') or (hand_type == 'high card'):
            best_hand = top_hands[0]
            for hand in top_hands[1:]:
                # Convert kickers (already sorted high to low) to integers and concatenate into one large number
                new_int = int("".join([str(ranks.index(kicker)) for kicker in hand.kickers]))
                best_int = int("".join([str(ranks.index(kicker)) for kicker in best_hand.kickers]))
                if new_int > best_int:
                    best_hand = hand
                else:
                    pass
        elif hand_type == 'full house': 
            best_hand = top_hands[0]
            for hand in top_hands[1:]:
                # Check trips
                if ranks.index(hand.high) > ranks.index(best_hand.high):
                    best_hand = hand
                elif ranks.index(hand.high) == ranks.index(best_hand.high):
                    if ranks.index(hand.low) > ranks.index(best_hand.low):
                        best_hand = hand
                    else:
                        pass
                else:
                    pass
        elif hand_type == '3-of-a-kind':
            best_hand = top_hands[0]
            for hand in top_hands[1:]:
                if ranks.index(hand.high) > ranks.index(best_hand.high):
                    best_hand = hand
                elif ranks.index(hand.high) == ranks.index(best_hand.high):
                    # Convert kickers (already sorted high to low) to integers and concatenate into one large number
                    new_int = int("".join([str(ranks.index(kicker)) for kicker in hand.kickers]))
                    best_int = int("".join([str(ranks.index(kicker)) for kicker in best_hand.kickers]))
                    if new_int > best_int:
                        best_hand = hand
                    else:
                        pass
                else:
                    pass
        elif hand_type == '2-pair':
            best_hand = top_hands[0]
            for hand in top_hands[1:]:
                if ranks.index(hand.high) > ranks.index(best_hand.high):
                    best_hand = hand
                elif ranks.index(hand.high) == ranks.index(best_hand.high):
                    if ranks.index(hand.low) > ranks.index(best_hand.low):
                        best_hand = hand
                    elif ranks.index(hand.low) == ranks.index(best_hand.low):
                        if ranks.index(hand.kickers[0]) > ranks.index(best_hand.kickers[0]):
                            best_hand = hand
                        else: pass
                    else:
                        pass
                else:
                    pass
        else: # pairs
            best_hand = top_hands[0]
            for hand in top_hands[1:]:
                if ranks.index(hand.high) > ranks.index(best_hand.high):
                    best_hand = hand
                elif ranks.index(hand.high) == ranks.index(best_hand.high):
                    # Convert kickers (already sorted high to low) to integers and concatenate into one large number
                    new_int = int("".join([str(ranks.index(kicker)) for kicker in hand.kickers]))
                    best_int = int("".join([str(ranks.index(kicker)) for kicker in best_hand.kickers]))
                    if new_int > best_int:
                        best_hand = hand
                    else:
                        pass
                else:
                    pass
        return best_hand
    
def find_winners(hands):
    '''Method to rank the top hand of each player, accounting for ties'''
    winning_value = max([x.value for x in hands])
    # If there is only one hand at the top, then return that as the winner
    winners = [x for x in hands if x.value == winning_value]
    if (len(winners) == 1) or (winners[0].name == 'royal flush'):
        return winners
    else:
        ranks = [str(n) for n in range(2, 11)] + list('JQKA')
        high_rank = ranks[max([ranks.index(x.high) for x in winners])]
        
        if 'straight' in winners[0].name:
            return [x for x in winners if x.high == high_rank]    
        elif winners[0].name == '4-of-a-kind':
            high_kicker = ranks[max([ranks.index(x.kickers[0]) for x in winners])]
            return [x for x in winners if (x.high == high_rank) and (x.kickers[0] == high_kicker)]   
        elif winners[0].name == 'full house':
            new_winners = [x for x in winners if x.high == high_rank]
            if len(new_winners) == 1:
                return new_winners
            else:
                best_low = ranks[max([ranks.index(x.low) for x in new_winners])]
                return [x for x in new_winners if x.low == best_low]     
        elif ((winners[0].name == 'flush') or (winners[0].name == '3-of-a-kind') or
              (winners[0].name == 'high card') or (winners[0].name == 'pair')):
            new_winners = [x for x in winners if x.high == high_rank]
            if len(new_winners) == 1:
                return new_winners
            else:
                totals = {}
                for i,hand in enumerate(new_winners):
                    # Convert kickers (already sorted high to low) to integers and concatenate into one large number
                    totals[i] = int("".join([str(ranks.index(kicker)) for kicker in hand.kickers]))
                best_total = max(totals.values())
                return [x for i,x in enumerate(new_winners) if totals[i] == best_total]   
        else: #2-pairs
            new_winners = [x for x in winners if x.high == high_rank]
            if len(new_winners) == 1:
                return new_winners
            else:
                best_low = ranks[max([ranks.index(x.low) for x in new_winners])]
                best_kicker = ranks[max([ranks.index(x.kickers[0]) for x in new_winners])]
                return [x for x in new_winners if (x.low == best_low) and (x.kickers[0] == best_kicker)]

## Simulate single game

In [6]:
%%time
game = 1
players = 2

# Create original, shuffled deck of 52 cards
deck = shuffle_deck(create_deck())

# Deal each player a hand
player_hands = deal_player_hands(deck, cards=2, players=players)

# Deal communal cards
communal_cards = deal_communal_cards(player_hands[1])

player_results = collections.namedtuple(
    'PlayerResult',
    ['game_num', 'num_players', 'player', 'hole_cards', 'best_hand', 'result'])

top_hands = {}
for player in range(players):
    top_hands[player + 1] = find_top_hand_for_player(combinations(player_hands[0][player].cards + communal_cards, 5))

winning_hands = find_winners(top_hands.values())    

game_results = []
for player in top_hands.keys():
    if top_hands[player] in winning_hands:
        if len(winning_hands) == 1:
            winloss = 'Win'
        else:
            winloss = 'Draw'
    else:
        winloss = 'Loss'
        
    game_results.append(player_results(
        game,
        players,
        player,
        player_hands[0][player - 1].name,
        top_hands[player].name,
        winloss))


Wall time: 52.3 ms


In [7]:
game_results

[PlayerResult(game_num=1, num_players=2, player=1, hole_cards='A-high', best_hand='high card', result='Loss'),
 PlayerResult(game_num=1, num_players=2, player=2, hole_cards='connectors', best_hand='straight', result='Win')]

## Simulate Multiple Games with Various Players

In [160]:
%%time
game_results = []
for players in range(2, 11):
    for game in range(10):
        # Create original, shuffled deck of 52 cards
        deck = shuffle_deck(create_deck())

        # Deal each player a hand
        player_hands = deal_player_hands(deck, cards=2, players=players)

        # Deal communal cards
        communal_cards = deal_communal_cards(player_hands[1])

        player_results = collections.namedtuple(
            'PlayerResult',
            ['game_num', 'num_players', 'player', 'hole_cards', 'best_hand', 'result'])

        top_hands = {}
        for player in range(players):
            top_hands[player + 1] = find_top_hand_for_player(combinations(player_hands[0][player].cards + communal_cards, 5))

        winning_hands = find_winners(top_hands.values())    

        for player in top_hands.keys():
            if top_hands[player] in winning_hands:
                if len(winning_hands) == 1:
                    winloss = 'Win'
                else:
                    winloss = 'Draw'
            else:
                winloss = 'Loss'

            game_results.append(player_results(
                game,
                players,
                player,
                player_hands[0][player - 1].name,
                top_hands[player].name,
                winloss))
        

Wall time: 13.4 s


In [161]:
game_results

[PlayerResult(game_num=0, num_players=2, player=1, hole_cards='suited', best_hand='2-pair', result='Win'),
 PlayerResult(game_num=0, num_players=2, player=2, hole_cards='connectors', best_hand='high card', result='Loss'),
 PlayerResult(game_num=1, num_players=2, player=1, hole_cards='Q-high', best_hand='3-of-a-kind', result='Win'),
 PlayerResult(game_num=1, num_players=2, player=2, hole_cards='pair', best_hand='2-pair', result='Loss'),
 PlayerResult(game_num=2, num_players=2, player=1, hole_cards='K-high', best_hand='pair', result='Loss'),
 PlayerResult(game_num=2, num_players=2, player=2, hole_cards='K-high', best_hand='2-pair', result='Win'),
 PlayerResult(game_num=3, num_players=2, player=1, hole_cards='suited', best_hand='high card', result='Loss'),
 PlayerResult(game_num=3, num_players=2, player=2, hole_cards='connectors', best_hand='pair', result='Win'),
 PlayerResult(game_num=4, num_players=2, player=1, hole_cards='6-high', best_hand='pair', result='Loss'),
 PlayerResult(game_nu

## Profiling

In [8]:
%load_ext line_profiler

ModuleNotFoundError: No module named 'line_profiler'